# Notebook to trigger full OPERA-RTC validation runs. 

- Select a validation module
- Select a site and orbital path combination
- Run validation module on all scenes, as listed in OPERA-RTC_CalVal_S1_Scene_IDs.csv
    - Downloads and mosaicks bursts
    - Runs validation workflows

In [ ]:
import os
from pathlib import Path
import subprocess
from typing import Union

from opensarlab_lib import work_dir

import ipywidgets as widgets
from ipywidgets import Layout

In [ ]:
calval_types = [
    
    # TODO don't hardcode, pull from CSV
    "Absolute Geolocation Evaluation",
    # "Gamma0 Comparison on Foreslope, Backslope, Flat Regions",
    "Coregistration",
    # "Regression Analysis"
]

calval_module = widgets.RadioButtons(
    options=calval_types,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Select an OPERA-RTC validation analysis to perform.")
display(calval_module)

In [ ]:
if "Absolute" in calval_module.value:
    
    # TODO don't hardcode, pull from CSV
    sites = [
        "California, Ascending Orbit, Orbital Path 64",
        "Oklahoma, Ascending Orbit, Orbital Path 34",
        "Oklahoma, Descending Orbit, Orbital Path 107",
    ]
    script = "bulk_papermill_OPERA_RTC_ALE.py"
elif calval_module.value == 'Coregistration':
    sites = [
        "Delta Junction, Ascending Orbit, Path 94",
        "Delta Junction, Descending Orbit, Path 160",
        "Brazil, Descending Orbit, Path 170"
    ]
    script = "bulk_papermill_OPERA_RTC_Coregistration.py"
else:
    raise Exception(f"{calval_module.value} analysis option not yet added to this notebook")
    
site_options = widgets.RadioButtons(
    options=sites,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Select a site/path for the validation analysis.")
display(site_options)

In [ ]:
download_mosaic = [
    'Download and mosaic OPERA sample burst data',
    'Validate previously downloaded and mosaicked data'
]

download_options = widgets.RadioButtons(
    options=download_mosaic,
    description='',
    disabled=False,
    layout=Layout(min_width="1000px")
    )

print("Download and mosaic OPERA sample burst data or validate previously prepared mosaics?")
display(download_options)

if calval_module.value == 'Coregistration':
    print("\nSave or delete intermediary data produced during validation?")
    delete_options = widgets.RadioButtons(
        options=['Save intermediary data', 'Delete intermediary data'],
        description='',
        disabled=False,
        layout=Layout(min_width="1000px")
        )
    display(delete_options)

In [ ]:
download = '--skip_download' if 'Validate' in download_options.value else ''
delete = '--delete_intermediary' if (calval_module.value == 'Coregistration' and 'delete_options' in locals() and 'Delete' in delete_options.value) else ''
script_dir = Path.cwd()/"bulk_validation_scripts"
with work_dir(script_dir):
    site = site_options.value.split(',')[0]
    orbital_path = site_options.value.split(' ')[-1]
    # print(f'python {script} --site "{site}" --orbital_path {orbital_path} {download} {delete}')
    subprocess.run([f'python {script} --site "{site}" --orbital_path {orbital_path} {download} {delete}'], shell=True)